In [1]:
#GOOD LUCK
import re
import numpy as np
import pandas as pd
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM
from keras.utils import to_categorical
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences


Using TensorFlow backend.


In [2]:
'''def pad_sequences(sequences, maxlen=2000):
    for s in sequences:
        if len(s) == 0: yield np.zeros((1, maxlen), dtype='int32')
        else:
          s = np.array(s, dtype='int32')
          yield np.expand_dims(np.pad(s[:maxlen], (0, max(0, maxlen - len(s))), 'constant'), axis=0)'''

"def pad_sequences(sequences, maxlen=2000):\n    for s in sequences:\n        if len(s) == 0: yield np.zeros((1, maxlen), dtype='int32')\n        else:\n          s = np.array(s, dtype='int32')\n          yield np.expand_dims(np.pad(s[:maxlen], (0, max(0, maxlen - len(s))), 'constant'), axis=0)"

In [3]:
path_slang = "datafile/abusive_slang_stemmed.txt"
path_religious = "datafile/abusive_religious_stemmed.txt"
path_personal_attack = "datafile/abusive_personal_attack_stemmed.txt"
path_antifaminism = "datafile/abusive_antifaminism_stemmed.txt"
path_political =  "datafile/abusive_political_stemmed.txt"


def read_file(path):
    sent =[]
    sent.clear()
    with open(path, "r",encoding="utf-8") as f:
        for line in f:
            sent.append(line)
    return sent
slang_sent = read_file(path_slang)
religious_sent = read_file(path_religious)
personal_attack_sent = read_file(path_personal_attack)
political_sent = read_file(path_political)
antifaminism_sent = read_file(path_antifaminism)


In [4]:
import re
def digit_remover(text):
    text = re.sub(r'[\d]','',text)
    text = re.sub(r'[?\!\t\‛\’\(\)\.\।]+','',text)
    text = re.sub(r'[/\-\,]',' ',text)
    text = text.rstrip("\n\r")
    return text


In [5]:
def remover(sent):
    for i in range(len(sent)):
        sent[i] = digit_remover(sent[i])
    return sent
slang_sent = remover(slang_sent)
religious_sent = remover(religious_sent)
personal_attack_sent = remover(personal_attack_sent)
political_sent = remover(political_sent)
antifaminism_sent = remover(antifaminism_sent)


In [6]:
slang_df = pd.DataFrame({'col':slang_sent})
slang_df['label'] = 0
religious_df = pd.DataFrame({'col':religious_sent})
religious_df['label'] = 1
personal_attack_df = pd.DataFrame({'col':personal_attack_sent})
personal_attack_df['label'] = 2
political_df = pd.DataFrame({'col':political_sent})
political_df['label'] = 3
antifaminism_df = pd.DataFrame({'col':antifaminism_sent})
antifaminism_df['label'] = 4

In [7]:
frames = [slang_df, religious_df,personal_attack_df,political_df,antifaminism_df]
result = pd.concat(frames,ignore_index=True)
#suffle the data frame
result = result.sample(frac=1)

In [50]:
from sklearn.model_selection import train_test_split
seed = 7
np.random.seed(seed)
X = result['col']
y = result['label']
y = to_categorical(result['label'], num_classes=5)
X_train,X_test,y_train,y_test = train_test_split(X,y,random_state = seed,test_size=.10)

In [51]:
tokenize = Tokenizer()
tokenize.fit_on_texts(X_train.values)

In [52]:
X_train = tokenize.texts_to_sequences(X_train)
X_test = tokenize.texts_to_sequences(X_test)

In [53]:
max_lenght = max([len(s.split()) for s in result['col']])
X_train = pad_sequences(X_train, max_lenght)
X_test = pad_sequences(X_test, max_lenght)

In [54]:
from imblearn.over_sampling import SMOTE
smote = SMOTE('minority')
X_sm,y_sm = smote.fit_sample(X_train,y_train)
print(X_sm.shape,y_sm.shape)

(2947, 124) (2947, 5)


In [55]:
#I choose to build 3 hidden layers
EMBEDDING_DIM = 100
unknown = len(tokenize.word_index)+1
model = Sequential()
model.add(Embedding(unknown, EMBEDDING_DIM, input_length=max_lenght))
model.add(LSTM(units=128, dropout=0.1, recurrent_dropout=0.2 ))
model.add(Dense(5, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [56]:
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 124, 100)          568900    
_________________________________________________________________
lstm_5 (LSTM)                (None, 128)               117248    
_________________________________________________________________
dense_5 (Dense)              (None, 5)                 645       
Total params: 686,793
Trainable params: 686,793
Non-trainable params: 0
_________________________________________________________________
None


In [57]:
print(y_train.shape)
print(X_train.shape)

(2248, 5)
(2248, 124)


In [58]:
#model.fit(X_sm, y_sm, batch_size=64, epochs=7, verbose=1)
model.fit(X_train, y_train, batch_size=64, epochs=10, verbose=1)

Epoch 1/10
2248/2248 [==============================] - 29s 13ms/step - loss: 1.5040 - acc: 0.4133
Epoch 2/10
2248/2248 [==============================] - 28s 13ms/step - loss: 1.3744 - acc: 0.4391
Epoch 3/10
2248/2248 [==============================] - 30s 13ms/step - loss: 1.0154 - acc: 0.6379
Epoch 4/10
2248/2248 [==============================] - 28s 13ms/step - loss: 0.6500 - acc: 0.7998
Epoch 5/10
2248/2248 [==============================] - 28s 13ms/step - loss: 0.3481 - acc: 0.9141
Epoch 6/10
2248/2248 [==============================] - 30s 13ms/step - loss: 0.1797 - acc: 0.9520
Epoch 7/10
2248/2248 [==============================] - 26s 12ms/step - loss: 0.1034 - acc: 0.9729
Epoch 8/10
2248/2248 [==============================] - 25s 11ms/step - loss: 0.0676 - acc: 0.9818
Epoch 9/10
2248/2248 [==============================] - 23s 10ms/step - loss: 0.0466 - acc: 0.9858
Epoch 10/10
2248/2248 [==============================] - 27s 12ms/step - loss: 0.0329 - acc: 0.9898


In [59]:
final_pred = model.predict_classes(X_test)
final_pred= to_categorical(final_pred)

In [60]:
# calculate accuracy of class predictions
from sklearn import metrics
metrics.accuracy_score(y_test, final_pred)

0.648

In [61]:
score,acc = model.evaluate(X_test,y_test,batch_size=64,verbose=1)
print(acc)

250/250 [==============================] - 1s 6ms/step
0.6480000066757202


In [62]:
# print the classification report
print(metrics.classification_report(y_test, final_pred))

              precision    recall  f1-score   support

           0       0.36      0.43      0.39        42
           1       0.88      0.62      0.73        45
           2       0.69      0.69      0.69       108
           3       0.93      0.81      0.86        31
           4       0.52      0.71      0.60        24

   micro avg       0.65      0.65      0.65       250
   macro avg       0.67      0.65      0.65       250
weighted avg       0.68      0.65      0.66       250
 samples avg       0.65      0.65      0.65       250

